# PEP-695: Type Parameter Syntax

Changes has been made to the syntax to make typing in python feel more first class. This include removing the need to import and explicitly declare a `TypeVar` or a `TypeVarTuple`.

### The new `TypeAlias`

Let's start with `TypeAlias`, a type alias can contain a `TypeVar` likes so:

In [11]:
from typing import TypeAlias, TypeVar, assert_type

T = TypeVar("T")

OldPair: TypeAlias = tuple[T, T]
assert_type((1, 1), OldPair[int])


(1, 1)

The new syntax completely removes the need for the `TypeVar`

In [12]:
type NewPair[T] = tuple[T, T]
assert_type((1, 1), NewPair[int])

(1, 1)

Syntax also added for more complex situations:

##### TypeVarTuple

In [13]:
from typing import Iterable, TypeVarTuple


# OLD
Ts = TypeVarTuple("Ts")
QueryResultOld: TypeAlias = Iterable[tuple[*Ts]]

# New
type QueryResult[*Ts] = Iterable[tuple[*Ts]]  

##### ParamSpec

In [14]:
from typing import Callable, ParamSpec


P = ParamSpec("P")

OldCastFunction: TypeAlias = Callable[[Callable[P, float]], Callable[P, int]]

type CastFunction[**P] = Callable[[Callable[P, float]], Callable[P, int]]

##### Bound/Constraints

In [16]:
from decimal import Decimal
from typing import Callable, Hashable, TypeAlias, TypeVar


# Bound
T = TypeVar("T", bound=Hashable)

HashFunctionOld: TypeAlias = Callable[[T], int]


type HashFunction[T: Hashable] = Callable[[T], int]


# Constraints
T = TypeVar("T", Decimal, int, float)

UnitConverterOld: TypeAlias = Callable[[T], T]

type UnitCoverter[T: (Decimal, int, float)] = Callable[[T], T] 


### Function Type Parameter

New syntax will work in generic functions, including using the bound/constraint syntax

In [17]:
from typing import Iterable, TypeVar, reveal_type

T = TypeVar("T", int, float, str)

def first(it: Iterable[T]) -> T | None:
    for element in it:
        return element
    
    return None


reveal_type(first(['a']))


def first[T: (int, float, str)](it: Iterable[T]) -> T | None:
    for element in it:
        return element
    
    return None

reveal_type(first(['a']))

Runtime type is 'str'
Runtime type is 'str'


'a'

### `Generics`

Likewise with generics:

In [10]:
from dataclasses import dataclass
from typing import Generic

T = TypeVar("T")


@dataclass
class Matix(Generic[T]):
    data: list[list[T]]

    def __getitem__(self, location: tuple[int, int]) -> T:
        return self.data[location[0]][location[1]]
    

@dataclass
class Matix[T]:
    data: list[list[T]]

    def __getitem__(self, location: tuple[int, int]) -> T:
        return self.data[location[0]][location[1]]
    

reveal_type(Matix([[1]]))

Runtime type is 'Matix'


Matix(data=[[1]])